In [1]:
import torch
import pandas as pd
import numpy as np
device = torch.device('cuda')

C:\Users\nikit\miniconda3\envs\rpz\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_fwf('train.csv', header=None)
data = df[0].str.split(',', expand=True).values.tolist()
len(data)

23547

In [3]:
from collections import Counter
cusine_counter = Counter()
cusine_vocab = {}

for recipe in data:
  if None in recipe:
    recipe_length = recipe.index(None) - 1
  else:
    recipe_length = len(recipe) - 1
  cusine_counter[recipe[recipe_length]] +=1

print(cusine_counter)

cusine_num = 0
for k, v in cusine_counter.items():
  if v > 100:
    cusine_vocab[k] = cusine_num
    cusine_num+=1

print(cusine_vocab)

Counter({'italian': 4661, 'mexican': 3782, 'southern_us': 2497, 'indian': 1702, 'chinese': 1581, 'french': 1534, 'cajun_creole': 900, 'thai': 886, 'japanese': 833, 'greek': 711, 'spanish': 584, 'moroccan': 489, 'british': 482, 'korean': 470, 'vietnamese': 468, 'filipino': 450, 'irish': 403, 'russian': 298, 'brazilian': 279, 'jamaican': 274, '': 44, '6': 19, '5': 16, '64': 6, 'mexi': 5, 'chines': 5, 'm': 5, 'i': 5, '671': 4, '62': 4, '55': 4, 'vi': 3, 'ital': 3, 'vietnames': 3, '52': 3, '4': 3, 'cajun_creol': 3, 's': 3, 'southern': 3, '61': 3, 'india': 2, 'mexic': 2, '553': 2, '6294': 2, '642': 2, 'cajun': 2, 't': 2, 'viet': 2, 'mex': 2, '495': 2, 'southern_u': 2, 'itali': 2, '5377': 2, 'chine': 2, '6422': 2, 'fre': 2, '53': 2, '63': 2, 'italia': 2, 'cajun_c': 2, 'vietna': 2, 'caju': 2, 'me': 2, '29': 1, 'brit': 1, '620': 1, '612': 1, '656': 1, 'southe': 1, 'cajun_cr': 1, '5774': 1, '65': 1, '618': 1, '6276': 1, 'japane': 1, '4162': 1, '5908': 1, '5884': 1, '658': 1, '630': 1, '588': 1,

In [4]:
# train_ingredients = []
# train_labels = []

# for recipe in data:
#   if None in recipe:
#     recipe_length = recipe.index(None) - 1
#   else:
#     recipe_length = len(recipe) - 1
#   if recipe[recipe_length] in cusine_vocab:
#     train_ingredients.append(list(map(int, recipe[:recipe_length])))
#     train_labels.append(cusine_vocab[recipe[recipe_length]])

# print(len(train_ingredients), len(train_labels))

Derrick's reading

In [5]:
#ingredient_name = pd.read_csv(os.getcwd() + '/project description/node_ingredient.txt', engine='python', delimiter=',,', header=None)
df = pd.read_csv('train.csv', engine= 'python', sep='\,',  names=list(range(61)))
df1 = df.fillna(0)
df_2 = df1.values.tolist()

In [6]:
# Separating the cuisines from the recipies 
train_ingredients = []
train_labels = []
for i, val in enumerate(df_2):
    R_l = [v for v in val if v !=0]
    train_ingredients.append(list(map(int, R_l[:-1]))) 
    train_labels.append(cusine_vocab[R_l[-1]])

In [7]:
print(train_ingredients[0])

[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187]


In [8]:
max_len = max(train_ingredients, key = lambda x: len(x))
print(len(max_len))

59


In [9]:
# df = pd.read_fwf('validation_classification_question.csv', header=None)
# val_x = df[0].str.split(',', expand=True).values.tolist()
# df = pd.read_fwf('validation_classification_answer.csv', header=None)
# val_y = df[0].str.split(',', expand=True).values.tolist()
# print(len(val_x), len(val_y))

In [10]:
df = pd.read_csv('validation_classification_question.csv', engine= 'python', sep='\,',  names=list(range(60)))
df1 = df.fillna(0)
val_x = df1.values.tolist()

df = pd.read_csv('validation_classification_answer.csv', engine= 'python', sep='\,',  names=list(range(60)))
df1 = df.fillna(0)
val_y = df1.values.tolist()

In [11]:
val_ingredients_c = []
val_labels_c = []

for i in range(len(val_x)):
  R_l = [v for v in val_x[i] if v !=0]
  val_ingredients_c.append(list(map(int, R_l[:-1]))) 
  val_labels_c.append(cusine_vocab[val_y[i][0]])

print(len(val_ingredients_c), len(val_labels_c))

7848 7848


In [12]:
# val_ingredients_c = []
# val_labels_c = []

# for i in range(len(val_x)):
#   if val_y[i][0] in cusine_vocab:
#     if None in val_x[i]:
#       recipe_length = val_x[i].index(None)
#     else:
#       recipe_length = len(val_x[i])
#     if recipe_length == 0:
#       print(123)
#     #print(val_x[i][:recipe_length])
#     #print(val_x[i])
#     if val_x[i][recipe_length-1] == "":
#       val_ingredients_c.append(list(map(int, val_x[i][:recipe_length-1]))) # without one ingredient ??
#     else:
#       val_ingredients_c.append(list(map(int, val_x[i][:recipe_length])))
#     val_labels_c.append(cusine_vocab[val_y[i][0]])

# print(len(val_ingredients_c), len(val_labels_c))

In [13]:
from torch.utils.data import Dataset as TorchDataset

class Dataset(TorchDataset):
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):

        input_tensor = torch.LongTensor([self.data[index]]) # the first dimension is minibatch size
        label = torch.LongTensor([self.labels[index]])
        return input_tensor, label

    def __len__(self):
        return len(self.data)

In [20]:
from torch import nn

class Transformer(nn.Module):
  def __init__(self, num_tokens, num_labels, dim_model, num_heads, num_encoder_layers, num_decoder_layers, dropout_p, use_pretrained_embeddings = False):
    super(Transformer, self).__init__()
    self.use_pretrained_embeddings = use_pretrained_embeddings
    if not use_pretrained_embeddings:
      self.embedding = nn.Embedding(num_tokens, dim_model)
    self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=1024,
            dropout=dropout_p,
        )
    self.linear = nn.Linear(dim_model, num_labels)

  def forward(self, input_tensor):
    if not self.use_pretrained_embeddings:
       x = self.embedding(input_tensor).squeeze(1)
    else:
      x = input_tensor
    # print(x.shape)
   # x = x.permute(1, 0, 2)
    transformer_out = self.transformer(x, x)
    #print(transformer_out.shape)
    out = self.linear(torch.mean(transformer_out, dim=1))
    #out = self.linear(transformer_out[:,0,:])
    return out

In [21]:
def train(model, criterion, optimizer, data_loader, validation_loader, epochs, clipping_on=False):
  model.to(device=device)
  for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch in data_loader:
      x = batch[0].to(device=device)
      #print(x[0])
      y = batch[1].flatten().to(device=device)
      preds = model(x)
      #print(preds.shape, y.shape)
      loss = criterion(preds, y)
      optimizer.zero_grad()
      loss.backward()
      if clipping_on:
        nn.utils.clip_grad_norm_(model.parameters(), 1)
      optimizer.step()
      running_loss += loss.item()

    epoch_loss = running_loss / len(data_loader)

    val_loss = 0.0
    model.eval()
    with torch.no_grad():
      for batch in validation_loader:
          
          x = batch[0].to(device=device)
          y = batch[1].flatten().to(device=device)
          preds = model(x)
          loss = criterion(preds, y)
          val_loss += loss.item()
      val_loss /= len(validation_loader)
        
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, epoch_loss, val_loss))
    #print(measure_accuracy(model, train_loader))
    print(measure_accuracy(model, validation_loader))

In [23]:
def pad_tensor(vec, length, dim, pad_symbol):
    # Credits to https://discuss.pytorch.org/t/dataloader-for-various-length-of-data/6418/8
    pad_size = list(vec.shape)
    #print(pad_size)
    pad_size[dim] = length - vec.size(dim)
    return torch.cat([vec, torch.zeros(*pad_size, dtype=torch.int64)], dim=dim)

class Padder:
    
    def __init__(self, dim=0, pad_symbol=0):
        self.dim = dim
        self.pad_symbol = pad_symbol
        
    def __call__(self, batch):
        max_len = max(map(lambda x: x[0].shape[self.dim], batch))
        batch = list(map(lambda xy:
                    (pad_tensor(xy[0], max_len, self.dim, self.pad_symbol), xy[1]), batch))
        xs = torch.stack(list(map(lambda x: x[0], batch)), dim=0)
        ys = torch.stack(list(map(lambda x: x[1], batch)), dim=0)
        return xs, ys

In [24]:
# batch_size = 256
# epochs = 20 # 40 0.7469

# num_labels = len(cusine_vocab)
# num_tokens = 6714
# dim_model = 128
# num_heads = 4
# num_encoder_layers = 2
# num_decoder_layers = 2
# dropout_p = 0.2

In [25]:
batch_size = 256
epochs = 20 # 40 0.7469

num_labels = len(cusine_vocab)
num_tokens = 6714
dim_model = 128
num_heads = 4
num_encoder_layers = 1
num_decoder_layers = 1
dropout_p = 0.3
# 1024 dim_feedforward

In [26]:
from torch.utils.data import DataLoader

padder = Padder(dim=1, pad_symbol=0)
train_dataset = Dataset(train_ingredients, train_labels)
validation_dataset = Dataset(val_ingredients_c, val_labels_c)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, collate_fn = padder, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, collate_fn = padder)

In [31]:
model = Transformer(num_tokens, num_labels, dim_model, num_heads, num_encoder_layers, num_decoder_layers, dropout_p)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [32]:
train(model, criterion, optimizer, train_loader, validation_loader, epochs)

Epoch: 0, Training Loss: 2.446950236092443, Validation Loss: 2.1439752155734646
tensor(0.4084, device='cuda:0')
Epoch: 1, Training Loss: 1.9776771548001661, Validation Loss: 1.7705618296900103
tensor(0.4864, device='cuda:0')
Epoch: 2, Training Loss: 1.692632601312969, Validation Loss: 1.5968618739035823
tensor(0.5299, device='cuda:0')
Epoch: 3, Training Loss: 1.502042117326156, Validation Loss: 1.4450606838349374
tensor(0.5631, device='cuda:0')
Epoch: 4, Training Loss: 1.368546178807383, Validation Loss: 1.3497509341086111
tensor(0.5961, device='cuda:0')
Epoch: 5, Training Loss: 1.2488208128058391, Validation Loss: 1.2714740576282624
tensor(0.6214, device='cuda:0')
Epoch: 6, Training Loss: 1.1665352926306103, Validation Loss: 1.2313473724549817
tensor(0.6340, device='cuda:0')
Epoch: 7, Training Loss: 1.074194552457851, Validation Loss: 1.1673733380533033
tensor(0.6470, device='cuda:0')
Epoch: 8, Training Loss: 1.012580236662989, Validation Loss: 1.1444685459136963
tensor(0.6478, device

In [29]:
def predict_on_batch(model, batch, multitask_switch=None):
  x = batch[0].to(device=device)
  #print(x)
  if multitask_switch == "cuisine":
    prediction, _ = model(x)
  elif multitask_switch == "ingredients":
    _, prediction = model(x)
  else:
    prediction = model(x)
  probs = nn.functional.softmax(prediction, dim=1)
  #print("Preds: ", torch.argmax(probs, axis=1))
  return probs

In [30]:
def measure_accuracy(model, data_loader, multitask_switch=None):
  correct_preds = 0
  model.eval()
  with torch.no_grad():
    for batch in data_loader:
        preds = predict_on_batch(model, batch, multitask_switch)
        target = batch[1].to(device=device).flatten()
      # print("Target: ", target)
        #print(torch.argmax(preds, axis=1).shape, target.shape)
        correct_preds += torch.sum(torch.argmax(preds, axis=1) == target)
    accuracy = correct_preds / len(data_loader.dataset)
  return accuracy

In [33]:
print(measure_accuracy(model, train_loader))
print(measure_accuracy(model, validation_loader))

tensor(0.8479, device='cuda:0')
tensor(0.7153, device='cuda:0')


# Word embeddings experiment

In [33]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import numpy as np

emb_length = 100
glove_vocab = {}
with open('glove.6B.{}d.txt'.format(emb_length), encoding='utf-8') as f:
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      glove_vocab[word] = coefs

In [ ]:
PAD_embedding = torch.zeros(emb_length)
UNK_embedding = np.mean(list(glove_vocab.values()), axis=0)

In [ ]:
df = pd.read_fwf('node_ingredient.csv', header=None)
node_ingredient = df[0].values.tolist()
print(len(node_ingredient))
ing_id_to_str = {i: ing for i,ing in enumerate(node_ingredient)}
print(ing_id_to_str)

6714
{0: 'coca-cola', 1: 'vegan butter', 2: 'sourdough rolls', 3: 'reduced sodium refried beans', 4: 'ramen noodles', 5: 'crumbled corn bread', 6: 'japanese breadcrumbs', 7: 'toasted shredded coconut', 8: 'chinese spinach', 9: "Hellmann's® Real Mayonnaise", 10: 'ducklings', 11: 'basil olive oil', 12: 'white baking bar', 13: 'rye whiskey', 14: 'mushroom broth', 15: 'meat loaf mix', 16: 'cocktail sauce', 17: 'asparagus spears', 18: 'nonfat greek yogurt', 19: 'cabernet sauvignon', 20: 'low sodium tomato sauce', 21: 'shredded colby', 22: 'dried dillweed', 23: 'frozen potatoes', 24: 'hot dog bun', 25: 'balm leaves', 26: 'lo mein noodles', 27: 'green split peas', 28: 'celery leaves', 29: 'butterflied leg of lamb', 30: 'refrigerated piecrusts', 31: 'cajun spice mix', 32: 'finely chopped onion', 33: 'chili bean sauce', 34: 'brown rice', 35: 'cutlet', 36: 'beef jerky', 37: 'lemon grass', 38: 'ground walnuts', 39: 'kinchay', 40: 'macarons', 41: 'crème de cassis', 42: 'tiger lily buds', 43: 'sour

In [ ]:
class EmbDataset(TorchDataset):

    def __init__(self, data, labels, vocab, unk_emb, ing_dict):
        self.data = data
        self.labels = labels
        self.vocab = vocab
        self.unk_emb = unk_emb
        self.ing_dict = ing_dict

    def __getitem__(self, index):

        input_embeddings = []
        for id in self.data[index]:
          ing_str = self.ing_dict[id]
          #print(ing_str)
          #print(ing_str)
          #ing_emb = [self.vocab[word] if word in self.vocab for word else self.unk_emb in ing_str.lower().split(" ")]
          ing_emb = []
          for word in ing_str.lower().split(" "):
            if word in self.vocab:
              ing_emb.append(self.vocab[word])
          if not ing_emb:
            ing_emb = [self.unk_emb]
          #print(ing_emb)
          ing_emb = np.mean(ing_emb, axis=0)
          #print(ing_emb.shape)
          input_embeddings.append(torch.Tensor(ing_emb))
        # input_embeddings = [torch.Tensor(self.vocab[word]) if word in self.vocab else self.unk_emb for word in self.ing_dict[self.data[index]]] # UNK if word not found
        input_tensor = torch.stack(input_embeddings, dim=1).T
        label = torch.LongTensor([self.labels[index]])
        return input_tensor, label

    def __len__(self):
        return len(self.data)

In [ ]:
from torch.utils.data import DataLoader

padder = Padder(dim=0, pad_symbol=PAD_embedding)
train_dataset = EmbDataset(train_ingredients, train_labels, glove_vocab, UNK_embedding, ing_id_to_str)
validation_dataset = EmbDataset(val_ingredients, val_labels, glove_vocab, UNK_embedding, ing_id_to_str)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, collate_fn = padder, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, collate_fn = padder)

In [ ]:
model = Transformer(num_tokens, num_labels, emb_length, num_heads, num_encoder_layers, 0, dropout_p, use_pretrained_embeddings = True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train(model, criterion, optimizer, train_loader, validation_loader, epochs=10)

Epoch: 0, Training Loss: 2.842891972997914, Validation Loss: 2.7060829997062683
Epoch: 1, Training Loss: 2.6269504609315293, Validation Loss: 2.59051650762558
Epoch: 2, Training Loss: 2.553463583407195, Validation Loss: 2.5362174808979034
Epoch: 3, Training Loss: 2.498174978339154, Validation Loss: 2.488337367773056
Epoch: 4, Training Loss: 2.4473945679871933, Validation Loss: 2.4392791986465454
Epoch: 5, Training Loss: 2.39649948866471, Validation Loss: 2.3903819918632507
Epoch: 6, Training Loss: 2.3431253122246782, Validation Loss: 2.3415041863918304
Epoch: 7, Training Loss: 2.2914845839790674, Validation Loss: 2.293892055749893
Epoch: 8, Training Loss: 2.2414082547892695, Validation Loss: 2.247974932193756
Epoch: 9, Training Loss: 2.192669630050659, Validation Loss: 2.203691750764847


In [ ]:
print(measure_accuracy(model, train_loader))
print(measure_accuracy(model, validation_loader))

tensor(0.4054, device='cuda:0')
tensor(0.3897, device='cuda:0')


In [ ]:
# 0.58 0.57 with unk, 0.6 0.59 without (10 epochs)
# 0.81 0.7430 40 epochs (worse than trained encoding)
# 0.8795 0.7630 60 epochs (no further improvement)

# Completion task

In [41]:
completion_data = []
completion_labels = []

for recipe in train_ingredients:
  ingredients_num = len(recipe)
  for i in range(ingredients_num):
    incomplete_recipe = recipe[:ingredients_num].copy()
    missing_ingredient = incomplete_recipe.pop(i)

    completion_data.append(incomplete_recipe)
    completion_labels.append(missing_ingredient)

print(len(completion_data), len(completion_labels))

253453 253453


In [42]:
max_len = max(completion_data, key = lambda x: len(x))
print(len(max_len))

58


In [43]:
df = pd.read_csv('validation_completion_question.csv', engine= 'python', sep='\,',  names=list(range(60)))
df1 = df.fillna(0)
val_x = df1.values.tolist()

df = pd.read_csv('validation_completion_answer.csv', engine= 'python', sep='\,',  names=list(range(60)))
df1 = df.fillna(0)
val_y = df1.values.tolist()

print(len(val_x), len(val_y))

7848 7848


In [44]:
val_ingredients = []
val_labels = []

for i in range(len(val_x)):
  R_l = [v for v in val_x[i] if v !=0]
  val_ingredients.append(list(map(int, R_l[:-1]))) 
  val_labels.append(int(val_y[i][0]))

print(len(val_ingredients), len(val_labels))

7848 7848


In [45]:
# val_ingredients = []
# val_labels = []

# for i in range(len(val_x)):
#   if None in val_x[i]:
#     recipe_length = val_x[i].index(None)
#   else:
#     recipe_length = len(val_x[i])
#   if val_x[i][recipe_length-1] == "":
#     val_ingredients.append(list(map(int, val_x[i][:recipe_length-1]))) # without one ingredient ??
#   else:
#     val_ingredients.append(list(map(int, val_x[i][:recipe_length])))
#   val_labels.append(val_y[i])

# print(len(val_ingredients), len(val_labels))

In [46]:
# import csv
# with open('completion_train.csv', 'w', newline='') as f:
      
#     # using csv.writer method from CSV package
#     write = csv.writer(f)
#     write.writerows(completion_data)

In [47]:
# import csv
# with open('completion_labels.csv', 'w', newline='') as f:
      
#     # using csv.writer method from CSV package
#     write = csv.writer(f)
#     write.writerows([completion_labels])

In [48]:
from torch.utils.data import DataLoader

padder = Padder(dim=1, pad_symbol=0)
train_dataset = Dataset(completion_data, completion_labels)
validation_dataset = Dataset(val_ingredients, val_labels)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, collate_fn = padder, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, collate_fn = padder)

In [49]:
print(train_dataset[1])

(tensor([[2813, 3229, 3885, 4379, 4390, 5250, 5456, 6187]]), tensor([3146]))


In [50]:
model = Transformer(num_tokens, num_tokens, dim_model, num_heads, num_encoder_layers, num_decoder_layers, dropout_p)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [51]:
train(model, criterion, optimizer, train_loader, validation_loader, epochs=20)

Epoch: 0, Training Loss: 6.538893952259022, Validation Loss: 6.410460518252465
tensor(0.0400, device='cuda:0')
Epoch: 1, Training Loss: 6.3007899254530395, Validation Loss: 6.182196509453558
tensor(0.0487, device='cuda:0')
Epoch: 2, Training Loss: 6.076858339107602, Validation Loss: 6.050140150131718
tensor(0.0570, device='cuda:0')
Epoch: 3, Training Loss: 5.924446343172691, Validation Loss: 5.928442016724618
tensor(0.0669, device='cuda:0')
Epoch: 4, Training Loss: 5.779839670622024, Validation Loss: 5.8316786366124305
tensor(0.0765, device='cuda:0')
Epoch: 5, Training Loss: 5.642403364422102, Validation Loss: 5.7189845269726165
tensor(0.0873, device='cuda:0')
Epoch: 6, Training Loss: 5.516332074924627, Validation Loss: 5.654345481626449
tensor(0.0937, device='cuda:0')
Epoch: 7, Training Loss: 5.393568569187199, Validation Loss: 5.580202671789354
tensor(0.1030, device='cuda:0')
Epoch: 8, Training Loss: 5.286091132794331, Validation Loss: 5.613425562458653
tensor(0.1028, device='cuda:0'

In [52]:
print(measure_accuracy(model, train_loader))
print(measure_accuracy(model, validation_loader))
# Prev version (wrong)
# 20 epcohs, 0.1745 0.1073 5.72(val loss) (0.1121)
# 25 epochs, 0.2093 0.1140 6.10(val loss)

#New version
# 8 epochs, 0.0948val
# 20 epochs, 0.1937 0.1242 (5.43 val loss)
# 34 epochs 0.1282 (5.63 loss)

tensor(0.2061, device='cuda:0')
tensor(0.1115, device='cuda:0')


# Mutlti-task experiments

In [16]:
# multitask_data = []
# multitask_labels = []

# for recipe in data:
#   if None in recipe:
#       ingredients_num = recipe.index(None) - 1
#   else:
#       ingredients_num = len(recipe) - 1
#   if recipe[ingredients_num] in cusine_vocab:
#     for i in range(ingredients_num):
#       incomplete_recipe = list(map(int, recipe[:ingredients_num].copy()))
#       #print(recipe, ingredients_num)
#       missing_ingredient = incomplete_recipe.pop(i)
#       #incomplete_recipe.insert(0, 6714 + cusine_vocab[recipe[ingredients_num]])

#       multitask_data.append(incomplete_recipe)
#       multitask_labels.append([cusine_vocab[recipe[ingredients_num]], missing_ingredient])

# print(len(multitask_data), len(multitask_labels))

246547 246547


In [53]:
multitask_data = []
multitask_labels = []


for recipe, cusine in zip(train_ingredients, train_labels):
  ingredients_num = len(recipe)
  for i in range(ingredients_num):
    incomplete_recipe = recipe[:ingredients_num].copy()
    #print(recipe, ingredients_num)
    missing_ingredient = incomplete_recipe.pop(i)
    #incomplete_recipe.insert(0, 6714 + cusine_vocab[recipe[ingredients_num]])

    multitask_data.append(incomplete_recipe)
    multitask_labels.append([cusine, missing_ingredient])
    
print(len(multitask_data), len(multitask_labels))

253453 253453


In [54]:
from torch import nn

class MiltitaskTransformer(nn.Module):
  def __init__(self, num_tokens, num_labels, dim_model, num_heads, num_encoder_layers, num_decoder_layers, dropout_p, use_pretrained_embeddings = False):
    super(MiltitaskTransformer, self).__init__()
    self.use_pretrained_embeddings = use_pretrained_embeddings
    if not use_pretrained_embeddings:
      self.embedding = nn.Embedding(num_tokens, dim_model)
    self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=1024,
            dropout=dropout_p,
        )
    self.linear_cuisine = nn.Linear(dim_model, num_labels)
    self.linear_ingredients = nn.Linear(dim_model, num_tokens)

  def forward(self, input_tensor):
    if not self.use_pretrained_embeddings:
       x = self.embedding(input_tensor).squeeze(1)
    else:
      x = input_tensor
    # print(x.shape)
   # x = x.permute(1, 0, 2)
    transformer_out = self.transformer(x, x)
    #print(transformer_out.shape)
    out_cuisine = self.linear_cuisine(torch.mean(transformer_out, dim=1))
    out_ingredients = self.linear_ingredients(torch.mean(transformer_out, dim=1))
    #out = self.linear(transformer_out[:,0,:])
    return out_cuisine, out_ingredients

In [55]:
import tqdm

def train_multitask(model, criterion, optimizer, data_loader, validation_loader_cuisine, validation_loader_ingredients, epochs, clipping_on=False):
  model.to(device=device)
  for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch in  tqdm.tqdm(data_loader):
      x = batch[0].to(device=device)
      #print(x[0])
      #print(batch[1].shape)
      y_cuisine = batch[1][:,:,0].flatten().to(device=device)
      y_ingredients = batch[1][:,:,1].flatten().to(device=device)
      preds_cuisine, preds_ingredients = model(x)
      #print(preds.shape, y.shape)
      loss = criterion(preds_cuisine, y_cuisine) + criterion(preds_ingredients, y_ingredients)
      optimizer.zero_grad()
      loss.backward()
      if clipping_on:
        nn.utils.clip_grad_norm_(model.parameters(), 1)
      optimizer.step()
      running_loss += loss.item()

    epoch_loss = running_loss / len(data_loader)

    val_loss_cuisine = 0.0
    val_loss_ingredients = 0.0
    model.eval()
    with torch.no_grad():
      for batch in validation_loader_cuisine:
          x = batch[0].to(device=device)
          y = batch[1].flatten().to(device=device)
          preds_cuisine, preds_ingredients = model(x)
          loss = criterion(preds_cuisine, y)
          val_loss_cuisine += loss.item()
            
      for batch in validation_loader_ingredients:
          x = batch[0].to(device=device)
          y = batch[1].flatten().to(device=device)
          preds_cuisine, preds_ingredients = model(x)
          loss = criterion(preds_ingredients, y)
          val_loss_ingredients += loss.item()
      val_loss_cuisine /= len(validation_loader_cuisine)
      val_loss_ingredients /= len(validation_loader_ingredients)
        
    print('Epoch: {}, Training Loss: {}, Validation Loss Cuisine: {}, Validation Loss Ingredients: {}'.format(epoch, epoch_loss, val_loss_cuisine, val_loss_ingredients))
    #print(measure_accuracy(model, train_loader))
    print("Validation classification accuracy: {}".format(measure_accuracy(model, validation_loader_cuisine, multitask_switch="cuisine")))
    print("Validation completion accuracy: {}".format(measure_accuracy(model, validation_loader_ingredients, multitask_switch="ingredients")))

In [56]:
from torch.utils.data import DataLoader

padder = Padder(dim=1, pad_symbol=-1)
train_dataset = Dataset(multitask_data, multitask_labels)
validation_dataset_cuisine = Dataset(val_ingredients_c, val_labels_c)
validation_dataset_ingredients = Dataset(val_ingredients, val_labels)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, collate_fn = padder, shuffle=True)
validation_loader_cuisine = DataLoader(dataset=validation_dataset_cuisine, batch_size=batch_size, collate_fn = padder)
validation_loader_ingredients = DataLoader(dataset=validation_dataset_ingredients, batch_size=batch_size, collate_fn = padder)

In [57]:
batch_size = 256
epochs = 20 # 40 0.7469

num_labels = len(cusine_vocab)
num_tokens = 6714
dim_model = 128
num_heads = 4
num_encoder_layers = 1
num_decoder_layers = 1
dropout_p = 0.3
# 1024 dim_feedforward

In [58]:
model = MiltitaskTransformer(num_tokens, num_labels, dim_model, num_heads, num_encoder_layers, num_decoder_layers, dropout_p)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [59]:
train_multitask(model, criterion, optimizer, train_loader, validation_loader_cuisine, validation_loader_ingredients, epochs=20)

100%|████████████████████████████████████████████████████████████████████████████████| 991/991 [01:09<00:00, 14.29it/s]


Epoch: 0, Training Loss: 7.9988337693854366, Validation Loss Cuisine: 1.3444204445808166, Validation Loss Ingredients: 6.106983584742392
Validation classification accuracy: 0.5939092636108398
Validation completion accuracy: 0.041029561311006546


100%|████████████████████████████████████████████████████████████████████████████████| 991/991 [01:09<00:00, 14.31it/s]


Epoch: 1, Training Loss: 6.910449245021755, Validation Loss Cuisine: 1.0774759188775094, Validation Loss Ingredients: 5.913303036843577
Validation classification accuracy: 0.6986492872238159
Validation completion accuracy: 0.0569571852684021


100%|████████████████████████████████████████████████████████████████████████████████| 991/991 [01:09<00:00, 14.34it/s]


...

Epoch: 17, Training Loss: 4.677319370774039, Validation Loss Cuisine: 1.368550464030235, Validation Loss Ingredients: 5.739656571419008
Validation classification accuracy: 0.7284657955169678
Validation completion accuracy: 0.11544342339038849


100%|████████████████████████████████████████████████████████████████████████████████| 991/991 [01:09<00:00, 14.27it/s]


Epoch: 18, Training Loss: 4.607863224118076, Validation Loss Cuisine: 1.3775022452877415, Validation Loss Ingredients: 5.766874697900588
Validation classification accuracy: 0.7279561161994934
Validation completion accuracy: 0.11404179036617279


100%|████████████████████████████████████████████████████████████████████████████████| 991/991 [01:09<00:00, 14.26it/s]


Epoch: 19, Training Loss: 4.539788564447197, Validation Loss Cuisine: 1.4195415300707663, Validation Loss Ingredients: 5.790287263931766
Validation classification accuracy: 0.7255351543426514
Validation completion accuracy: 0.1153159961104393


In [ ]:
#75.8 and 14.1 (11 epochs: 75.5, 13.6)

In [173]:
import gc

model = None
gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()